In [ ]:
import os
import re
import pandas as pd

In [ ]:
corpus_path = 'path/to/your/data/JA/Data' ###
out_path = './output_dir/'
vl = True  #set to True if you're using the Flemish part
country = 'vl' if vl else 'nl'

In [3]:
def process_utterance(utter, file_id, spk_id):
    res = file_id + ' 1 '
    if utter[2][0] != '"':
        return ';;' + spk_id + '\n'
    
    if utter[2][1] == '"':
        res += 'inter_segment_gap ' + utter[0][0:-1] + ' ' + utter[1][0:-1] + ' <o,f0,>'
    else:
        text = re.sub(r'\*.', '', utter[2][1:-2])
        text = re.sub('ggg', '', text)
        text = re.sub('xxx', '', text)
        text = re.sub('Xxx', '', text)
        if text == '\n':
            res += 'inter_segment_gap ' + utter[0][0:-1] + ' ' + utter[1][0:-1] + ' <o,f0,>'
        else:
            res += spk_id + ' ' + utter[0][0:-1] + ' ' + utter[1][0:-1] + ' <o,f1,unknown> ' + text

    return res + '\n'

In [ ]:
df = pd.read_excel(os.path.join(corpus_path, 'data/meta/xls', country, 'recordings.xls'))

for _, row in df.iterrows():
    ort_path = os.path.join(corpus_path, 'data/annot/text/ort', row["Component"], country, row["Root"] + '.ort')
    output_dir = os.path.join(out_path, row["Component"], country)
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, row["Root"] + '.stm')

    with open(ort_path, encoding="latin_1") as infile, open(output_path, 'w', encoding="utf-8") as outfile:
        lines = infile.readlines()
        id_spk = ''
        utter = []

        for line in lines:
            if re.match(r'"N0|"N1|"V000', line):
                id_spk = line[1:-2]
                continue
            if id_spk == '':
                continue
            if line[1:-2] in ("IntervalTier", "TextTier"):
                break

            if len(utter) < 3:
                utter.append(line)
                continue
            else:
                processed_utter = process_utterance(utter, row["Root"], row["SpeakerID"])
                outfile.write(processed_utter)
                utter = [line]
